## Dynamic Plans

Here, instead of a predefined plan, vehicles take decisions online following a minimization distance rule. Only a goal is predefined.

In [1]:
import numpy as np


# Custom modules
from helpers import global2local, plot_3d_interactive, kill_processes
from simulators import Simulator, Gazebo, QGC
from plan import Plan, State,ActionNames
from env import Enviroment


from helpers.navegation_logic import find_path, find_best_waypoint


kill_processes()

## Gonfiguration

In [2]:
offsets = [ #east, north, up, heading
    (40, -10, 0, 0), 
    (40,  20, 0, 0)]
n_vehicles = len(offsets)

wps = np.array([(40, -10, 5),
                (15, -10, 5),
                (0, -10, 5),
                (0,   4, 5),
                (40, 20, 5),
                (15, 20, 5),
                (0,  20, 5)])

homes=np.array([offset[:3] for offset in offsets])
target_wps=np.array([
                (0, 4,5),
                (0, 4,5)])

targets=np.array([(*target_wp[:2],0) for target_wp in target_wps])
plans=[None,None]

## Visualization Parameters

In [3]:
markers = {'waypoints': {'pos':wps,'color':'yellow'},
           'homes': {'pos':homes,'color':'blue'},
           'targets':{'pos':targets,'color':'green'}}

plot_3d_interactive(markers,title='Simulation Markers',expand=[0.2,0.2,0.4],ground=-0.05)

## Waypoint selection algorithm

The path is shown in local coordinates

In [4]:
local_wps=global2local(wps, homes,pairwise=False)
local_target_wps=global2local(target_wps, homes,pairwise=True)


for i,(target_wp,wps) in enumerate(zip(local_target_wps,local_wps)):
        local_path=find_path(np.array((0,0,0)),target_wp, wps,eps=1)
        print(f'uav-{i} Best path: {local_path}') 


uav-0 Best path: [[  0   0   0]
 [  0   0   5]
 [-25   0   5]
 [-40   0   5]
 [-40  14   5]]
uav-1 Best path: [[  0   0   0]
 [  0   0   5]
 [-25   0   5]
 [-40   0   5]
 [-40 -16   5]]


## Choose Simulator

In [5]:
#simulator = Simulator(name="NONE",offsets=offsets,plans=plans)

# simulator=QGC(offsets=offsets,
#               plans=plans,
#               origin=(-35.3633245,149.1652241,0,0))

simulator=Gazebo(offsets=offsets,
                 plans=plans,
                 world_path="simulators/gazebo/worlds/small_city.world",
                 vehicle_models=n_vehicles*["drone"],
                 markers=markers)

Launch Simulator

In [6]:
uavs = simulator.launch()
env = Enviroment(uavs)

Vehicle 1 launched 🚀
Vehicle 2 launched 🚀


## Execute Plan 

This should be simplified

In [7]:
from plan.actions import make_go_to

In [8]:
while env.vehs:
    for uav in env.vehs.copy():
        curr_action=uav.current_action()
        if curr_action.name==ActionNames.FLY and (curr_action.state == State.NOT_STARTED or (curr_action.state == State.IN_PROGRESS and curr_action.current.state == State.DONE)):
            final_wp=local_target_wps[uav.sys_id-1]
            curr_pos=uav.current_position()
            dist = np.linalg.norm(curr_pos - final_wp)
            if dist > 0.5:
                next_wp = find_best_waypoint(curr_pos, final_wp, local_wps[uav.sys_id-1],eps=1)
                next_step=make_go_to(wp=next_wp,wp_margin=0.5,cause_text="(dynamic)")
                next_step.bind_connection(uav.conn)
                uav.plan.current.add_next(next_step)
        if uav.plan.state == State.DONE:
            env.remove(uav)
        else:
            uav.plan.act()
kill_processes()

Vehicle 2: ▶️ Starting Step: Check disarmed
Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 2: ✅ Step: Check disarmed is done
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 2: ▶️ Starting Step: Check EKF
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 2: ✅ Step: Check EKF is done
Vehicle 2: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check GPS
Vehicle 2: ✅ Step: Check GPS is done
Vehicle 2: ▶️ Starting Step: Check system
Vehicle 1: ✅ Step: Check GPS is done
Vehicle 1: ▶️ Starting Step: Check system
Vehicle 2: ✅ Step: Check system is done
Vehicle 2: ▶️ Starting Step: Switch to GUIDED
Vehicle 2: ✅ Step: Switch to GUIDED is done
Vehicle 2: ▶️ Starting Step: arm
Vehicle 1: ✅ Step: Check system is done
Vehicle 1: ▶️ Starting Step: Switch to GUIDED
Vehicle 1: ✅ Step: Switch to GUIDED is done
Vehicle 1: ▶️ Starting Step: arm
Vehicle 1: ✅ Step: arm is done
Vehicle 1: ▶️ Starting Step: takeoff
Vehicle 1:📍 Distance to target: 5.01 m
Vehicle